# OBJECTIVE 1
Write a single function that takes in a filepath as input and saves the 8 rotated images.

In [5]:
# Code Sources:
# (1) https://stackoverflow.com/a/10404957
# (2) ChatGPT 3.5

import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as pyplot

# VERY DANGEROUS NEXT STEP
# We are doing this because the high-res image is supremely huge
#########################################
Image.MAX_IMAGE_PIXELS = None # DANGEROUS
#########################################

image_path = "CytAssist_FFPE_Protein_Expression_Human_Glioblastoma_image.tif"
high_res_path = "CytAssist_FFPE_Protein_Expression_Human_Glioblastoma_tissue_image.tif"

def make_grayscale(filepath):
    img = Image.open(filepath)
    img.getdata()
    r, g, b = img.split()
    ra, ga, ba = np.array(r), np.array(g), np.array(b)
    grayscale_as_list = ((0.299 * ra) + (0.587 * ga) + (0.114 * ba))
    return grayscale_as_list

def print_grayscale(grayscale_values):
    pyplot.figure()
    pyplot.imshow(grayscale_values, cmap="gray")
    
def grayscale_to_image(grayscale_values):
    height, width = len(grayscale_values), len(grayscale_values[0])
    img = Image.new("L", (width, height))

    # Iterate through the nested list and set pixel values
    for y in range(height):
        for x in range(width):
            pixel_value = round(grayscale_values[y][x])
            img.putpixel((x, y), pixel_value)

    return img

def mse(image1, image2):
    # Convert images to numpy arrays
    array1 = np.array(image1)
    array2 = np.array(image2)
    
    # Compute the Mean Squared Error (MSE)
    mse_value = np.mean((array1 - array2) ** 2)
    
    return mse_value

def mse_big_small(fidpath, hrpath):
    fid = Image.open(fidpath)
    fid.getdata()
    fid_red_data = np.array(fid.split()[0])
    fid_height, fid_width = len(fid_red_data), len(fid_red_data[0])
    
    hr = Image.open(hrpath)
    hr = hr.resize((fid_width, fid_height))
    return mse(make_grayscale(fidpath), hr)

def g2g(fidpath, hrpath):
    fid = Image.open(fidpath)
    hr = Image.open(hrpath)
    hr = hr.resize(fid.size)
    return mse(fid, hr)

def image_production(filepath):
    grayscale_image = grayscale_to_image(make_grayscale(filepath))
    mirrored_grayscale = grayscale_image.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
    
    # Get the directory path of the Jupyter notebook
    notebook_dir = os.path.dirname(os.path.abspath("__file__"))
    
    # Create a folder with the same name as the filepath
    folder_name = os.path.join(notebook_dir, os.path.splitext(os.path.basename(filepath))[0])
    os.makedirs(folder_name, exist_ok=True)
    
    for degree in [0, 90, 180, 270]:
        rotated_image = grayscale_image.rotate(degree)
        new_os_pathname = os.path.basename(filepath)[:-4] + f"_original_r{degree}.tif"
        rotated_image_path = os.path.join(folder_name, new_os_pathname)
        
        rotated_mirrored_image = mirrored_grayscale.rotate(degree)
        new_os_pathname = os.path.basename(filepath)[:-4] + f"_mirrored_r{degree}.tif"
        rotated_mirrored_image_path = os.path.join(folder_name, new_os_pathname)
        
        rotated_image.save(rotated_image_path)
        rotated_mirrored_image.save(rotated_mirrored_image_path)
        
        print(f"_original_r{degree} MSE: " + str(g2g(rotated_image_path, high_res_path)))
        print(f"_mirrored_r{degree} MSE: " + str(g2g(rotated_mirrored_image_path, high_res_path)))
        
    print(f"Rotated images saved at: {filepath[:-4]}")

In [6]:
%time image_production(image_path)

_original_r0 MSE: 23231.480255555554
_mirrored_r0 MSE: 23314.403870888887
_original_r90 MSE: 23565.038776222224
_mirrored_r90 MSE: 22877.622678
_original_r180 MSE: 23074.251782666666
_mirrored_r180 MSE: 23053.899898
_original_r270 MSE: 22958.110666444445
_mirrored_r270 MSE: 23508.58991911111
Rotated images saved at: CytAssist_FFPE_Protein_Expression_Human_Glioblastoma_image
CPU times: user 18.5 s, sys: 341 ms, total: 18.8 s
Wall time: 18.9 s


In [ ]:
# Code Credit: ChatGPT 3.5
# Rotations and saving

import os
from PIL import Image

### CHANGEABLE
image_path = "CytAssist_FFPE_Protein_Expression_Human_Glioblastoma_image.tif"

def rotate_and_save(filepath):
    # Get the directory path of the Jupyter notebook
    notebook_dir = os.path.dirname(os.path.abspath("__file__"))
    
    # Create a folder with the same name as the filepath
    folder_name = os.path.join(notebook_dir, os.path.splitext(os.path.basename(filepath))[0])
    os.makedirs(folder_name, exist_ok=True)
    
    # Load the image
    try:
        image = Image.open(filepath)
    except FileNotFoundError:
        print("File not found.")
        return
    
    for degree in [0, 90, 180, 270]:
        rotated_image = image.rotate(degree)
        new_os_pathname = os.path.basename(filepath)[:-4] + f"_r{degree}.tif"
        rotated_image_path = os.path.join(folder_name, new_os_pathname)
        
        rotated_image.save(rotated_image_path)
    
    print(f"Rotated images saved at: {filepath}")

# Example usage:
rotate_and_save(image_path)

In [1]:
# Code Sources:
# (1) https://stackoverflow.com/a/10404957
# (2) ChatGPT 3.5

import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import matplotlib.pyplot as pyplot

# VERY DANGEROUS NEXT STEP
# We are doing this because the high-res image is supremely huge
#########################################
Image.MAX_IMAGE_PIXELS = None # DANGEROUS
#########################################

def crop_and_save_tif(input_path, output_path):
    """
    Takes only the center 2400px by 2400px of the image
    """
    # Load the image
    image = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)

    # Get the dimensions of the image
    height, width = image.shape[:2]

    # Calculate the coordinates for cropping
    start_x = (width - 2400) // 2
    start_y = (height - 2400) // 2
    end_x = start_x + 2400
    end_y = start_y + 2400

    # Crop the image
    cropped_image = image[start_y:end_y, start_x:end_x]

    # Save the cropped image as a .tif file
    cv2.imwrite(output_path, cropped_image)

def make_grayscale(filepath):
    img = Image.open(filepath)
    img.getdata()
    r, g, b = img.split()
    ra, ga, ba = np.array(r), np.array(g), np.array(b)
    grayscale_as_list = ((0.299 * ra) + (0.587 * ga) + (0.114 * ba))
    return grayscale_as_list

def print_grayscale(grayscale_values):
    pyplot.figure()
    pyplot.imshow(grayscale_values, cmap="gray")
    
def grayscale_to_image(grayscale_values):
    height, width = len(grayscale_values), len(grayscale_values[0])
    img = Image.new("L", (width, height))

    # Iterate through the nested list and set pixel values
    for y in range(height):
        for x in range(width):
            pixel_value = round(grayscale_values[y][x])
            img.putpixel((x, y), pixel_value)

    return img

def mse(image1, image2):
    # Convert images to numpy arrays
    array1 = np.array(image1)
    array2 = np.array(image2)
    
    # Compute the Mean Squared Error (MSE)
    mse_value = np.mean((array1 - array2) ** 2)
    
    return mse_value

def mse_big_small(fidpath, hrpath):
    fid = Image.open(fidpath)
    fid.getdata()
    fid_red_data = np.array(fid.split()[0])
    fid_height, fid_width = len(fid_red_data), len(fid_red_data[0])
    
    hr = Image.open(hrpath)
    hr = hr.resize((fid_width, fid_height))
    return mse(make_grayscale(fidpath), hr)

def g2g(fidpath, hrpath):
    fid = Image.open(fidpath)
    hr = Image.open(hrpath)
    hr = hr.resize(fid.size)
    return mse(fid, hr)

def image_production(filepath):
    grayscale_image = grayscale_to_image(make_grayscale(filepath))
    mirrored_grayscale = grayscale_image.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
    
    # Get the directory path of the Jupyter notebook
    notebook_dir = os.path.dirname(os.path.abspath("__file__"))
    
    # Create a folder with the same name as the filepath
    folder_name = os.path.join(notebook_dir, os.path.splitext(os.path.basename(filepath))[0])
    os.makedirs(folder_name, exist_ok=True)
    
    new_path_names = []
    
    for degree in [0, 90, 180, 270]:
        rotated_image = grayscale_image.rotate(degree)
        new_os_pathname = os.path.basename(filepath)[:-4] + f"_original_r{degree}.tif"
        rotated_image_path = os.path.join(folder_name, new_os_pathname)
        new_path_names.append(rotated_image_path)
        
        rotated_mirrored_image = mirrored_grayscale.rotate(degree)
        new_os_pathname = os.path.basename(filepath)[:-4] + f"_mirrored_r{degree}.tif"
        rotated_mirrored_image_path = os.path.join(folder_name, new_os_pathname)
        new_path_names.append(rotated_mirrored_image_path)
        
        rotated_image.save(rotated_image_path)
        rotated_mirrored_image.save(rotated_mirrored_image_path)
        
    print(f"Rotated images saved at: {filepath[:-4]}")
    return new_path_names

def find_most_similar_image(large_image_path, small_images_paths):
    # Load the first small image to get its size
    sample_small_image = cv2.imread(small_images_paths[0], cv2.IMREAD_GRAYSCALE)

    # Load the large image
    large_image = cv2.imread(large_image_path, cv2.IMREAD_GRAYSCALE)

    # Resize the large image to match the size of the small image
    large_image_resized = cv2.resize(large_image, (sample_small_image.shape[1], sample_small_image.shape[0]))

    # Initialize variables to store the best match and its similarity score
    best_match = None
    best_score = -1

    # Iterate through each small image
    for small_image_path in small_images_paths:
        print("starting...")
        # Load the small image
        small_image = cv2.imread(small_image_path, cv2.IMREAD_GRAYSCALE)

        # Calculate the structural similarity index
        score, _ = ssim(large_image_resized, small_image, full=True)

        # If the similarity score is higher than the current best score, update the best match
        if score > best_score:
            best_score = score
            best_match = small_image_path
        print("finished 1!")

    return best_match, best_score

In [2]:
def find_orientation(fid_filepath: str, hr_filepath: str):
    crop_and_save_tif(fid_filepath, "temp_cropped_fid.tif")
    little_files = image_production("temp_cropped_fid.tif")
    return find_most_similar_image(hr_filepath, little_files)

In [4]:
fiducial_frame_path = "CytAssist_FFPE_Protein_Expression_Human_Glioblastoma_image.tif"
high_res_path = "CytAssist_FFPE_Protein_Expression_Human_Glioblastoma_tissue_image.tif"

%time find_orientation(fiducial_frame_path, high_res_path)

Rotated images saved at: temp_cropped_fid
starting...
finished 1!
starting...
finished 1!
starting...
finished 1!
starting...
finished 1!
starting...
finished 1!
starting...
finished 1!
starting...
finished 1!
starting...
finished 1!
CPU times: user 15.1 s, sys: 1.12 s, total: 16.2 s
Wall time: 16.2 s


('/Users/chengjunyuan/Documents/NUS Work/Y2S2 (Current)/Spatial Omics Hackathon/code/temp_cropped_fid/temp_cropped_fid_mirrored_r0.tif',
 0.020399975024691087)

In [ ]:
fiducial_frame_path = "CytAssist_FFPE_Protein_Expression_Human_Glioblastoma_image.tif"
image_path = "CytAssist_FFPE_Protein_Expression_Human_Glioblastoma_image.tif"
high_res_path = "CytAssist_FFPE_Protein_Expression_Human_Glioblastoma_tissue_image.tif"

# Example usage:
input_path = fiducial_frame_path
output_path = "output_image.tif"

crop_and_save_tif(input_path, output_path)